In [133]:
import pandas as pd
import math as m
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import numpy as np
from tqdm import tqdm_notebook
from sklearn import  metrics
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, log_loss, f1_score
from sklearn.linear_model import LogisticRegression, LinearRegression
%matplotlib inline

In [135]:
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
import lightgbm as lgbm
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module '_catboost' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
train = pd.DataFrame.from_csv('AHAN_TRAIN.csv')
test = pd.DataFrame.from_csv('AHAN_TEST.csv')
train.drop('LTTIME', axis=1, inplace=True)
test.drop('LTTIME', axis=1, inplace=True)

In [3]:
def get_data_model(train, n, target_col = 'target'):
    if n==1:
        print ('Already exists')
    else:
        train["target"+'_'+str(n)] = np.nan
        train.loc[:,"target"+'_'+str(n)] = np.append(train.target[n-1:].values,[np.nan]*(n-1))

In [188]:
def predict(train, model_type):
    for i in range(2, 19):
        get_data_model(train, n=i)
    target_col = ["target"+'_'+str(x) for x in range(1,19)]
    target_col[0] = 'target'
    estimaters = []
    
    for i in range(18):
        model = model_type#RandomForestClassifier(max_depth=3)
        model.fit(train[list(train.columns[:-18])+[target_col[i]]].dropna().drop(target_col[i], axis=1),
                  train[target_col[i]].dropna())
        estimaters.append(model)
        
    pred_month = []
    for day in range(1, 19):
        pred_day = []
        for i in range(day-1, 18):
            pred_day.append(estimaters[i].predict_proba(train.iloc[[-(i+1)]].drop(target_col, axis=1))[0][0])
        pred_day = np.array(pred_day)
        pred_month.append(pred_day)
    pred_month = np.array(pred_month)
    
    preds = []
    for i in range(18):
        preds.append((pred_month[i].mean().round()))
    return preds, pred_month

In [193]:
pred, month = predict(train[:], XGBClassifier())

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [194]:
accuracy_score(test.target,pred)

0.61111111111111116

In [195]:
print("Predictions: ", pred)
print("Target: ", test.target.values)

Predictions:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Target:  [0 0 0 0 1 0 1 1 0 0 0 0 1 0 0 0 1 0]


In [196]:
f1_score(test.target,pred)

0.36363636363636359